# Testing notebook

In [ ]:
from src.seq2seq.dataset import SeqDataset, pad_batch
from functools import partial

pad_batch_with_fixed_length = partial(pad_batch, fixed_length=128)
dataset_path = '/home/gkulemeyer/Documents/Repos/AEseq2seq/data/ArchiveII-KFold/common/fold_0_train.csv'
data = SeqDataset( dataset_path, min_len=0, max_len=512, verbose=False, cache_path=None, for_prediction=False,  training=False)

In [3]:
for n in range(3):
    print(n)
    print(data[n]['length'])
    print(data[n]['embedding'].shape)

0
119
torch.Size([4, 119])
1
120
torch.Size([4, 120])
2
120
torch.Size([4, 120])


In [15]:
from torch.utils.data import DataLoader

loader = DataLoader(data, batch_size=1, shuffle=False,collate_fn=pad_batch_with_fixed_length)

next(iter(loader))['embedding'].shape

torch.Size([1, 4, 128])

In [16]:
import torch as tr
x = tr.tensor([[
    [1,0,0,0,1,0,0,0],
    [0,1,0,0,0,1,0,0],
    [0,0,1,0,0,0,1,0],
    [0,0,0,1,0,0,0,1],]
])
x.shape

torch.Size([1, 4, 8])

Dado x, quiero `swap(x,prob)` o bien `swap(x,n_swap)`.
Teniendo en cuenta que `max_L` = 128,
Armo vector `swap_bool` shape:[4,127],
Armo vector `max_swap` 64.

In [22]:
x[0,:,7]

tensor([0, 0, 0, 1])